In [1]:
import os
import string
import annoy

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import codecs

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle

In [2]:
#pip install gensim

# Предобработаем данные для обучения классификатора

### Предобработаем датасет с товарами

In [3]:
# Загрузим датасет с наименованиями товаров
data = pd.read_csv('ProductsDataset.csv')
data.head(5)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35548 entries, 0 to 35547
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           35548 non-null  object 
 1   descrirption    33537 non-null  object 
 2   product_id      35536 non-null  object 
 3   category_id     35536 non-null  float64
 4   subcategory_id  35536 non-null  object 
 5   properties      35536 non-null  object 
 6   image_links     35533 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.9+ MB


In [5]:
data.isna().sum()

title                0
descrirption      2011
product_id          12
category_id         12
subcategory_id      12
properties          12
image_links         15
dtype: int64

In [6]:
# Создадим дополнительный датасет из пары описание-id товара, 
# чтобы далее его конкотенировать к основному датасету из пары заголовок-id товара
data_descr = data[['descrirption', 'product_id']].copy()
data_descr = data_descr.rename(columns={'descrirption': 'question'})
data_descr = data_descr.dropna()
data_descr

,question,product_id
0,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82
1,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34
2,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06
3,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027
4,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022
...,...,...
35542,"Привезли из Европы, не подошло по размеру\n\nР...",5ac078a7938000715c0e2c38
35543,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472
35544,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0
35545,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2


In [7]:
data = data.rename(columns={'title': 'question'})
data.head(5)

,question,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [8]:
data_title = pd.concat([data[['question', 'product_id']], data_descr[['question', 'product_id']]]).reset_index(drop=True)
data_title

,question,product_id
0,Юбка детская ORBY,58e3cfe6132ca50e053f5f82
1,Ботильоны,5667531b2b7f8d127d838c34
2,Брюки,59534826aaab284cba337e06
3,Продам детские шапки,57de544096ad842e26de8027
4,Блузка,5ad4d2626c86cb168d212022
...,...,...
69077,"Привезли из Европы, не подошло по размеру\n\nР...",5ac078a7938000715c0e2c38
69078,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472
69079,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0
69080,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2


In [9]:
# Переименуем и выделим только нужные нам стобцы title и product_id
data_title =data_title[['question', 'product_id']].rename(columns={'product_id': 'answer'})
data_title

,question,answer
0,Юбка детская ORBY,58e3cfe6132ca50e053f5f82
1,Ботильоны,5667531b2b7f8d127d838c34
2,Брюки,59534826aaab284cba337e06
3,Продам детские шапки,57de544096ad842e26de8027
4,Блузка,5ad4d2626c86cb168d212022
...,...,...
69077,"Привезли из Европы, не подошло по размеру\n\nР...",5ac078a7938000715c0e2c38
69078,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472
69079,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0
69080,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2


In [10]:
# Добавим колонку с признаком, что это запрос по товару
data_title['is_product'] = 1
data_title.head(5)

,question,answer,is_product
0,Юбка детская ORBY,58e3cfe6132ca50e053f5f82,1
1,Ботильоны,5667531b2b7f8d127d838c34,1
2,Брюки,59534826aaab284cba337e06,1
3,Продам детские шапки,57de544096ad842e26de8027,1
4,Блузка,5ad4d2626c86cb168d212022,1


# Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста

In [11]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open("prepared_answers.txt", "w", "utf-8") as fout:
    with codecs.open("Otvety.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [12]:
with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    data_answers = pd.Series(f)
data_answers

0          \tвопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧ...
1          Как парни относятся к цветным линзам? Если у д...
2          Что делать, сегодня нашёл 2 миллиона рублей? ....
3          Эбу в двенашке называется Итэлма что за эбу? ....
4          академия вампиров. сколько на даный момент час...
                                 ...                        
1163337    Между словами ПРЕЗИДЕНТ и РЕЗИДЕНТ есть что ли...
1163338    "Если это мое, то оно никуда от меня не денетс...
1163339    А Вы халяву любите или совесть имеете???) .\tЛ...
1163340    Так много разных гороскопов кто-нибудь может п...
1163341    В пылу страстей она сломала мне вставную челюс...
Length: 1163342, dtype: object

In [13]:
data_answers[0]

'\tвопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . \n'

In [16]:
def q_a_to_lst(data):
    '''
    Функция разделяет вопрос и ответ из текстового файла
    '''
    result = []
    for i in tqdm(data):
        spls = i.split("\t")
        result.append(spls)
    return result

In [17]:
data_answers = pd.DataFrame(q_a_to_lst(data_answers), columns=['question', 'answer'])
data_answers

  0%|          | 0/1163342 [00:00<?, ?it/s]

,question,answer
0,,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :). \n
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали..."
3,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...
4,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."
...,...,...
1163337,Между словами ПРЕЗИДЕНТ и РЕЗИДЕНТ есть что ли...,"Конечно, есть. Оба эти слова пришли из латыни,..."
1163338,"""Если это мое, то оно никуда от меня не денетс...","Если плохо обращаться со СВОЕЙ собакой, она сд..."
1163339,А Вы халяву любите или совесть имеете???) .,"Люблю, но нет её.. \n"
1163340,Так много разных гороскопов кто-нибудь может п...,[ссылка заблокирована по решению администрации...


In [18]:
data_answers['is_product'] = 0
data_answers

,question,answer,is_product
0,,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,0
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :). \n,0
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали...",0
3,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...,0
4,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть...",0
...,...,...,...
1163337,Между словами ПРЕЗИДЕНТ и РЕЗИДЕНТ есть что ли...,"Конечно, есть. Оба эти слова пришли из латыни,...",0
1163338,"""Если это мое, то оно никуда от меня не денетс...","Если плохо обращаться со СВОЕЙ собакой, она сд...",0
1163339,А Вы халяву любите или совесть имеете???) .,"Люблю, но нет её.. \n",0
1163340,Так много разных гороскопов кто-нибудь может п...,[ссылка заблокирована по решению администрации...,0


По нашим данных видно, что длина датасета с вопросами гораздо длинее, чем датасет с товарами. Попробуем сделать простой случайный сэмплинг для датасета с вопросами, чтобы сбалансировать данные по классам для обучения модели.

In [19]:
data_answers_sample = data_answers.sample(len(data_title))
print(data_answers_sample.shape)

(69082, 3)


Обьединим датасеты в один

In [20]:
data_train = pd.concat([data_title, data_answers_sample]).reset_index(drop=True)
data_train

,question,answer,is_product
0,Юбка детская ORBY,58e3cfe6132ca50e053f5f82,1
1,Ботильоны,5667531b2b7f8d127d838c34,1
2,Брюки,59534826aaab284cba337e06,1
3,Продам детские шапки,57de544096ad842e26de8027,1
4,Блузка,5ad4d2626c86cb168d212022,1
...,...,...,...
138159,"Как отказать мужчине в сексе, чтобы не отбить ...","В настоящее время на ""вооружении"" женщин есть ...",0
138160,НАЦИЯ которую ВСЕ НЕочень??? .,"евреи естественно, во все века гонимые и пресл...",0
138161,Кто это? .,Miriam Shor. \n,0
138162,"Боярченков, Михаил Александрович.",<p>Михаил Александрович Боярченков (9 ноября 1...,0


In [21]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    spls = ", ".join(spls)
    return spls

In [22]:
# Предбработаем колонку с question, удаляем знаки препинания и делаем лемматизацию
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

data_train['question'] = data_train['question'].apply(lambda x: preprocess_txt(x))
data_train

,question,answer,is_product
0,"юбка, детский, orby",58e3cfe6132ca50e053f5f82,1
1,ботильон,5667531b2b7f8d127d838c34,1
2,брюки,59534826aaab284cba337e06,1
3,"продать, детский, шапка",57de544096ad842e26de8027,1
4,блузка,5ad4d2626c86cb168d212022,1
...,...,...,...
138159,"отказать, мужчина, секс, отбить, желание, ухаж...","В настоящее время на ""вооружении"" женщин есть ...",0
138160,"нация, неочень","евреи естественно, во все века гонимые и пресл...",0
138161,,Miriam Shor. \n,0
138162,"боярченков, михаил, александрович",<p>Михаил Александрович Боярченков (9 ноября 1...,0


# Обучим классификатор

Подготовим данные для обучения

In [23]:
X = data_train['question']
y = data_train['is_product']

In [24]:
# Разделим данные на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [25]:
# Сделаем векторизацию текста
tfidf_vec = TfidfVectorizer().fit(X_train.values)
X_train_tfidf = tfidf_vec.transform(X_train)
X_test_tfidf = tfidf_vec.transform(X_test)

In [26]:
# Сохраним модель в файл и загрузим: 
with open('ChatBotPractice_tfidf.pkl', 'wb') as output:
    pickle.dump(tfidf_vec, output) #Сохраняем

with open('ChatBotPractice_tfidf.pkl', 'rb') as pkl_file:
    model_tfidf_vec = pickle.load(pkl_file) #Загружаем

#### LogisticRegression

In [27]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [28]:
predictions = model.predict(X_test_tfidf)

In [29]:
print('Accuracy LogisticRegression: ', accuracy_score(y_test, predictions))

Accuracy LogisticRegression:  0.9667426627582962


#### LinearSVC

In [30]:
model2 = LinearSVC()
model2.fit(X_train_tfidf, y_train)

LinearSVC()

In [31]:
predictions2 = model2.predict(X_test_tfidf)

In [32]:
print('Accuracy LinearSVC: ', accuracy_score(y_test, predictions2))

Accuracy LinearSVC:  0.9749936669923642


#### RandomForestClassifier

In [33]:
model3 = RandomForestClassifier()
model3.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [34]:
predictions3 = model3.predict(X_test_tfidf)

In [35]:
print('Accuracy RandomForestClassifier: ', accuracy_score(y_test, predictions3))

Accuracy RandomForestClassifier:  0.9659827018420005


In [36]:
# Сохраним модель LinearSVC в файл и загрузим: 
with open('ChatBotPractice_LinearSVC.pkl', 'wb') as output:
    pickle.dump(model3, output) #Сохраняем

with open('ChatBotPractice_LinearSVC.pkl', 'rb') as pkl_file:
    model_LinearSVC = pickle.load(pkl_file) #Загружаем

#### Проверим результаты классификации загрузив модели

In [37]:
question1 = 'юбка'

vec = model_tfidf_vec.transform([question1])
model_LinearSVC.predict(vec)[0]

1

In [38]:
question2 = 'как же так'
vec = model_tfidf_vec.transform([question2])
model_LinearSVC.predict(vec)[0]

0

In [39]:
question3 = 'шарф'
vec = model_tfidf_vec.transform([question3])
model_LinearSVC.predict(vec)[0]

1

In [40]:
question4 = 'В каком году'
vec = model_tfidf_vec.transform([question4])
model_LinearSVC.predict(vec)[0]

0

In [41]:
def get_class(question):
    vec = model_tfidf_vec.transform([question])
    question_class = model_LinearSVC.predict(vec)[0]
    return question_class

# Предобработаем текст в болталке, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [42]:
def preprocess_txt_speak(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [43]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with codecs.open("Otvety.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt_speak(line)
        sentences.append(spls)
        c += 1
        if c > 200000:
        #if c > 50000:
            break

0it [00:00, ?it/s]

In [44]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model")

In [45]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

for q, a in tqdm(zip(data_answers['question'], data_answers['answer'])):
    n_w2v = 0
    index_map[counter] = a
    question = preprocess_txt_speak(q)
    vector = np.zeros(100)
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index.add_item(counter, vector)

    counter += 1
    if counter > 20000:
        break

index.build(10)
index.save('speaker.ann')

0it [00:00, ?it/s]

True

In [46]:
def find_answer_speak(question):
    preprocessed_question = preprocess_txt_speak(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

In [47]:
# test
find_answer_speak('Почему пьяному везет больше чем трезвому')

'Бог на Руси бережет пьяных и дураков.... \n'

# Предобработаем текст в продуктовом датасете, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [49]:
sentences_pr = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
#c = 0


for question in tqdm(data_title['question']):
    spls = preprocess_txt_speak(question)
    sentences_pr.append(spls)
#     c += 1
#     #if c > 500000:
#     if c > 50000:
#        break

  0%|          | 0/69082 [00:00<?, ?it/s]

In [50]:
# Обучим модель word2vec на наших вопросах
sentences_pr = [i for i in sentences_pr if len(i) > 2]
model_pr = Word2Vec(sentences=sentences_pr, vector_size=100, min_count=1, window=5)
model_pr.save("w2v_pr_model")

In [51]:
index_product = annoy.AnnoyIndex(100 ,'angular')

index_pr = {}
counter = 0

for q, a in tqdm(zip(data_title['question'], data_title['answer'])):
    n_w2v = 0
    index_pr[counter] = a
    question = preprocess_txt_speak(q)
    vector = np.zeros(100)
    for word in question:
        if word in model_pr.wv:
            vector += model_pr.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    index_product.add_item(counter, vector)

    counter += 1
#     if counter > 50000:
#         break

index_product.build(1)
index_product.save('product.ann')

0it [00:00, ?it/s]

True

In [52]:
def find_answer_product(question):
    preprocessed_question = preprocess_txt_speak(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model_pr.wv:
            vector += model_pr.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index_product.get_nns_by_vector(vector, 1)
    return index_pr[answer_index[0]]

In [53]:
# Проверка... ну такое себе
find_answer_product('Женская зимняя куртка')

'5837eebcd9f65a3d91eeaef4'

In [58]:
data_title.loc[data_title['answer']=='5837eebcd9f65a3d91eeaef4']

,question,answer,is_product
1277,Зимняя женская куртка,5837eebcd9f65a3d91eeaef4,1
36764,"Очень качественная, фирменная. Р.54-56. В идеа...",5837eebcd9f65a3d91eeaef4,1


# Реализуем Чатбот

In [55]:
def get_answer(question):
    
    # Определяем класс вопроса 1-продукт, 0-болталка
    question_class = get_class(str(question))
    
    
    # Ответ 
    if question_class == 1:
        answer = find_answer_product(str(question))
    
    # Болтаем
    else:
        answer = find_answer_speak(str(question))
        
    return answer

# Тестируем

In [56]:
assert(get_answer('Юбка детская ORBY').startswith('58e3cfe6132ca50e053f5f82')), 'Не корректный код товара!'

In [57]:
assert(not get_answer('Где ключи от танка').startswith('5')) 